<a href="https://colab.research.google.com/github/DhaniAAA/Scrapping-X/blob/main/Scrapping%20X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Wajib Install
# Install library Python yang dibutuhkan
!pip install selenium pandas webdriver-manager

# Download dan install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb

# Jika ada error dependensi, jalankan perintah ini untuk memperbaikinya
!apt-get install -f

ERROR: Operation cancelled by user
--2025-06-09 15:50:03--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.2.136, 142.251.2.190, 142.251.2.91, ...
Connecting to dl.google.com (dl.google.com)|142.251.2.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117334784 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.1’

google-chrome-stabl 100%[===================>] 111.90M   139MB/s    in 0.8s    

2025-06-09 15:50:05 (139 MB/s) - ‘google-chrome-stable_current_amd64.deb.1’ saved [117334784/117334784]

(Reading database ... 126267 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (137.0.7151.68-1) over (137.0.7151.68-1) ...
Setting up google-chrome-stable (137.0.7151.68-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2

In [10]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import datetime

# --- KONFIGURASI ---
# SALIN DAN TEMPEL NILAI `auth_token` ANDA DI BAWAH INI.
# JANGAN BAGIKAN SCRIPT INI DENGAN COOKIE ANDA KARENA BERSIFAT RAHASIA.
AUTH_TOKEN_COOKIE = "Ganti dengan punya kalian"

# Waktu tunggu (dalam detik) antara setiap scroll agar halaman sempat memuat
SCROLL_PAUSE_TIME = 3
# --------------------


def setup_driver():
    """Menyiapkan instance WebDriver untuk Chrome."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        return driver
    except (WebDriverException, ValueError) as e:
        print("Error saat menyiapkan WebDriver.")
        print("Jika di Google Colab, pastikan sel instalasi Chrome sudah dijalankan.")
        print(f"Detail error: {e}")
        return None

def scrape_tweets(driver, query, target_count):
    """
    Mengekstrak data tweet dari halaman pencarian hingga target jumlah tweet tercapai.
    """
    # --- Login menggunakan Cookie ---
    print("Mengunjungi x.com untuk menyuntikkan cookie login...")
    driver.get("https://x.com")
    time.sleep(2)

    if AUTH_TOKEN_COOKIE and AUTH_TOKEN_COOKIE != "Ganti dengan Cookie kalian":
        cookie = {
            'name': 'auth_token', 'value': AUTH_TOKEN_COOKIE, 'domain': '.x.com',
            'path': '/', 'secure': True, 'httpOnly': True
        }
        driver.add_cookie(cookie)
        print("Cookie berhasil disuntikkan.")
    else:
        print("PERINGATAN: Cookie tidak diatur. Script mungkin akan terhadang halaman login.")

    # --- Memulai Pencarian ---
    search_url = f"https://x.com/search?q={query}&src=typed_query&f=live"
    print(f"Mengunjungi halaman pencarian: {search_url}")
    driver.get(search_url)

    try:
        print("Menunggu konten tweet muncul (maksimal 20 detik)...")
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//article[@data-testid='tweet']"))
        )
        print("Konten tweet terdeteksi. Memulai proses pengambilan data.")
    except TimeoutException:
        print("Batas waktu menunggu habis. Tidak ada tweet yang ditemukan.")
        print("Penyebab umum: Nilai cookie salah/kadaluarsa atau perubahan dari pihak X.")
        driver.save_screenshot('debug_screenshot.png')
        print("Screenshot 'debug_screenshot.png' telah disimpan untuk diperiksa.")
        return []

    # --- Scroll hingga target tercapai ---
    tweets_data = {} # Menggunakan dictionary untuk menyimpan data unik berdasarkan URL
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0

    while len(tweets_data) < target_count:
        print(f"\nTweet terkumpul: {len(tweets_data)}/{target_count}. Melakukan scroll...")

        tweet_articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")

        for tweet in tweet_articles:
            try:
                tweet_url_elements = tweet.find_elements(By.XPATH, ".//a[contains(@href, '/status/')]")
                tweet_url = tweet_url_elements[0].get_attribute('href') if tweet_url_elements else None

                if tweet_url and tweet_url not in tweets_data:
                    username = tweet.find_element(By.XPATH, ".//div[@data-testid='User-Name']//span").text
                    handle = tweet.find_element(By.XPATH, ".//span[contains(text(), '@')]").text
                    timestamp = tweet.find_element(By.XPATH, ".//time").get_attribute('datetime')
                    tweet_text = tweet.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text.replace('\n', ' ')
                    reply_count = tweet.find_element(By.XPATH, ".//button[@data-testid='reply']").text or "0"
                    retweet_count = tweet.find_element(By.XPATH, ".//button[@data-testid='retweet']").text or "0"
                    like_count = tweet.find_element(By.XPATH, ".//button[@data-testid='like']").text or "0"

                    tweets_data[tweet_url] = {
                        "username": username, "handle": handle, "timestamp": timestamp,
                        "tweet_text": tweet_text, "url": tweet_url, "reply_count": reply_count,
                        "retweet_count": retweet_count, "like_count": like_count
                    }
            except Exception:
                continue

        if len(tweets_data) >= target_count:
            print(f"Target {target_count} tweet telah tercapai. Menghentikan scroll.")
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scroll_attempts += 1
            print("Sudah mencapai akhir halaman...")
            if scroll_attempts > 3:
                print("Tidak ada tweet baru yang dimuat. Menghentikan proses.")
                break
        else:
            scroll_attempts = 0
        last_height = new_height

    return list(tweets_data.values())[:target_count]

def get_user_input():
    """Meminta input dari pengguna untuk parameter pencarian."""
    keyword = input("1. Masukkan kata kunci/topik pencarian: ")

    while True:
        try:
            target_count = int(input("2. Berapa jumlah tweet yang ingin diambil? "))
            if target_count > 0:
                break
            else:
                print("Jumlah harus lebih dari 0.")
        except ValueError:
            print("Input tidak valid, masukkan angka.")

    while True:
        start_date_str = input("3. Masukkan tanggal mulai (YYYY-MM-DD): ")
        try:
            datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Format tanggal salah. Gunakan format YYYY-MM-DD.")

    while True:
        end_date_str = input("4. Masukkan tanggal selesai (YYYY-MM-DD): ")
        try:
            datetime.datetime.strptime(end_date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Format tanggal salah. Gunakan format YYYY-MM-DD.")

    lang = input("5. Masukkan kode bahasa (misal: 'id' untuk Indonesia, 'en' untuk Inggris): ")

    query = f"{keyword} lang:{lang} until:{end_date_str} since:{start_date_str}"

    return query, target_count

def main():
    """Fungsi utama untuk menjalankan proses scraping."""
    if not AUTH_TOKEN_COOKIE or AUTH_TOKEN_COOKIE == "Ganti dengan Cookie kalian":
        print("Error: Harap isi variabel AUTH_TOKEN_COOKIE dengan nilai cookie Anda.")
        return

    search_query, target_tweet_count = get_user_input()

    driver = setup_driver()
    if not driver:
        return

    try:
        scraped_data = scrape_tweets(driver, search_query, target_tweet_count)

        if not scraped_data:
            print("Tidak ada data yang berhasil diambil. Coba periksa 'debug_screenshot.png' jika ada.")
            return

        df = pd.DataFrame(scraped_data)
        safe_keyword = "".join(c for c in search_query.split(' ')[0] if c.isalnum())
        output_filename = f"tweets_{safe_keyword}.csv"
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print(f"\n--- PROSES SELESAI ---")
        print(f"Data telah disimpan di file: {output_filename}")
        print(f"Total tweet yang berhasil diambil: {len(df)}")
        print("\nContoh data:")
        print(df.head())

    finally:
        if driver:
            print("\nMenutup browser...")
            driver.quit()

    # PERBAIKAN: Tambahkan pesan penutup yang jelas di akhir fungsi main.
    # Ini memastikan pengguna tahu bahwa program telah selesai sepenuhnya.
    print("\nEksekusi skrip telah selesai. Program sekarang akan berhenti.")


if __name__ == "__main__":
    main()


1. Masukkan kata kunci/topik pencarian: prabowo
2. Berapa jumlah tweet yang ingin diambil? 10
3. Masukkan tanggal mulai (YYYY-MM-DD): 2025-01-01
4. Masukkan tanggal selesai (YYYY-MM-DD): 2025-02-01
5. Masukkan kode bahasa (misal: 'id' untuk Indonesia, 'en' untuk Inggris): id
Mengunjungi x.com untuk menyuntikkan cookie login...
Cookie berhasil disuntikkan.
Mengunjungi halaman pencarian: https://x.com/search?q=prabowo lang:id until:2025-02-01 since:2025-01-01&src=typed_query&f=live
Menunggu konten tweet muncul (maksimal 20 detik)...
Konten tweet terdeteksi. Memulai proses pengambilan data.

Tweet terkumpul: 0/10. Melakukan scroll...

Tweet terkumpul: 5/10. Melakukan scroll...
Target 10 tweet telah tercapai. Menghentikan scroll.

--- PROSES SELESAI ---
Data telah disimpan di file: tweets_prabowo.csv
Total tweet yang berhasil diambil: 10

Contoh data:
                username            handle                 timestamp  \
0                Santoso   @Santoso6376854  2025-01-31T23:59:48.000Z

In [13]:
df = pd.read_csv("/content/tweets_Anies.csv")
df

,username,handle,timestamp,tweet_text,url,reply_count,retweet_count,like_count
0,Azwar Rozali,@azwar6102,2025-01-31T23:47:57.000Z,"Jadi ingat ketika Anies Baswedan nyapres, para...",https://x.com/azwar6102/status/188547513550080...,0,0,0
1,Azwar Rozali,@azwar6102,2025-01-31T23:32:54.000Z,Begitu muncul banyak masalah baru Anies Baswed...,https://x.com/azwar6102/status/188547134782199...,0,2,14
2,Azwar Rozali,@azwar6102,2025-01-31T23:17:31.000Z,Justru beliau yang menentang keras pembatalan ...,https://x.com/azwar6102/status/188546747767051...,0,0,0
3,D4NT3,@G0LIATH87,2025-01-31T13:25:02.000Z,Kehebatan Anies Baswedan,https://x.com/G0LIATH87/status/188531837580472...,2,0,0
4,kla,@_concave,2025-01-31T12:39:48.000Z,Klo seberangnya rumah dinas wapres itu baru ru...,https://x.com/_concave/status/1885306990093844618,1,0,1
...,...,...,...,...,...,...,...,...
95,KBA NEWS,@kba_news,2025-01-27T14:17:08.000Z,"Anies Baswedan Hadiri Milad ke-33 Al-Itqon, Be...",https://x.com/kba_news/status/1883881933123535129,0,0,1
96,KBA NEWS,@kba_news,2025-01-27T14:16:20.000Z,"Menatap 2029, Relawan P24 Silaturahmi dengan A...",https://x.com/kba_news/status/1883881733680242936,0,0,0
97,La Masia,@OxLaMasia,2025-01-27T11:03:39.000Z,Kamu yang pinter dan hebat masak ga bisa menga...,https://x.com/OxLaMasia/status/188383324346592...,1,0,0
98,Remita Kafiyah,@remita_kafiyah,2025-01-27T11:00:59.000Z,Lebih asik pilih Capres Ganjar Pranowo daripad...,https://x.com/remita_kafiyah/status/1883832569...,0,0,0
